## Evaluating a Classification Model

In this exercise, you will create a pipeline for a classification model, and then apply commonly used metrics to evaluate the resulting classifier.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load the source data
csv = spark.read.csv('wasb:///data/flights.csv', inferSchema=True, header=True)

# Select features and label
data = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("label")))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1528428190678_0006,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### Define the Pipeline and Train the Model
Now define a pipeline that creates a feature vector and trains a classification model

In [2]:
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [3]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+-------------------------------+----------+---------+
|features                       |prediction|trueLabel|
+-------------------------------+----------+---------+
|[1.0,1.0,10140.0,11259.0,0.0]  |0.0       |0        |
|[1.0,1.0,10140.0,11292.0,4.0]  |0.0       |0        |
|[1.0,1.0,10140.0,11298.0,-10.0]|0.0       |0        |
|[1.0,1.0,10140.0,11298.0,-10.0]|0.0       |0        |
|[1.0,1.0,10140.0,12264.0,2.0]  |0.0       |0        |
|[1.0,1.0,10140.0,12266.0,838.0]|1.0       |1        |
|[1.0,1.0,10140.0,12889.0,-4.0] |0.0       |0        |
|[1.0,1.0,10140.0,12889.0,-2.0] |0.0       |0        |
|[1.0,1.0,10140.0,12889.0,0.0]  |0.0       |0        |
|[1.0,1.0,10140.0,12892.0,26.0] |0.0       |1        |
|[1.0,1.0,10140.0,12892.0,37.0] |0.0       |1        |
|[1.0,1.0,10140.0,13204.0,32.0] |0.0       |1        |
|[1.0,1.0,10140.0,13232.0,-4.0] |0.0       |0        |
|[1.0,1.0,10140.0,14107.0,-9.0] |0.0       |0        |
|[1.0,1.0,10140.0,14107.0,-3.0] |0.0       |0        |
|[1.0,1.0,

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [4]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()

+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|            18877.0|
|       FP|               75.0|
|       TN|           649598.0|
|       FN|           142185.0|
|Precision| 0.9960426340227945|
|   Recall|0.11720331300989681|
+---------+-------------------+

### View the Raw Prediction and Probability
The prediction is based on a raw prediction score that describes a labelled point in a logistic function. This raw prediction is then converted to a predicted label of 0 or 1 based on a probability vector that indicates the confidence for each possible label value (in this case, 0 and 1). The value with the highest confidence is selected as the prediction.

In [5]:
prediction.select("rawPrediction", "probability", "prediction", "trueLabel").show(100, truncate=False)

+----------------------------------------+-----------------------------------------+----------+---------+
|rawPrediction                           |probability                              |prediction|trueLabel|
+----------------------------------------+-----------------------------------------+----------+---------+
|[1.5397427943555424,-1.5397427943555424]|[0.8234273319785481,0.17657266802145194] |0.0       |0        |
|[1.4842080811408414,-1.4842080811408414]|[0.8152073452953749,0.1847926547046251]  |0.0       |0        |
|[1.6789229651955888,-1.6789229651955888]|[0.8427618612771235,0.15723813872287645] |0.0       |0        |
|[1.6789229651955888,-1.6789229651955888]|[0.8427618612771235,0.15723813872287645] |0.0       |0        |
|[1.514769173086101,-1.514769173086101]  |[0.8197669228469342,0.18023307715306583] |0.0       |0        |
|[-10.111472793343712,10.111472793343712]|[4.060930203363316E-5,0.9999593906979662]|1.0       |1        |
|[1.5999775398079197,-1.5999775398079197]|[0.8

Note that the results include rows where the probability for 0 (the first value in the **probability** vector) is only slightly higher than the probability for 1 (the second value in the **probability** vector). The default *discrimination threshold* (the boundary that decides whether a probability is predicted as a 1 or a 0) is set to 0.5; so the prediction with the highest probability is always used, no matter how close to the threshold.

### Review the Area Under ROC
Another way to assess the performance of a classification model is to measure the area under a ROC curve for the model. the spark.ml library includes a **BinaryClassificationEvaluator** class that you can use to compute this. The ROC curve shows the True Positive and False Positive rates plotted for varying thresholds.

In [6]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print "AUR = ", aur

AUR =  0.923056709964

### Change the Discrimination Threshold
The AUC score seems to indicate a reasonably good model, but the performance metrics seem to indicate that it predicts a high number of False Negative labels (i.e. it predicts 0 when the true label is 1), leading to a low Recall. You can affect the way a model performs by changing its parameters. For example, as noted previously, the default discrimination threshold is set to 0.5 - so if there are a lot of False Positives, you may want to consider raising this; or conversely, you may want to address a large number of False Negatives by lowering the threshold.

In [7]:
lr2 = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3, threshold=0.35)
pipeline2 = Pipeline(stages=[assembler, lr2])
model2 = pipeline2.fit(train)
newPrediction = model2.transform(test)
newPrediction.select("rawPrediction", "probability", "prediction", "trueLabel").show(100, truncate=False)

+----------------------------------------+-----------------------------------------+----------+---------+
|rawPrediction                           |probability                              |prediction|trueLabel|
+----------------------------------------+-----------------------------------------+----------+---------+
|[1.5397427943555426,-1.5397427943555426]|[0.8234273319785481,0.1765726680214519]  |0.0       |0        |
|[1.4842080811408414,-1.4842080811408414]|[0.8152073452953749,0.1847926547046251]  |0.0       |0        |
|[1.678922965195589,-1.678922965195589]  |[0.8427618612771235,0.15723813872287642] |0.0       |0        |
|[1.678922965195589,-1.678922965195589]  |[0.8427618612771235,0.15723813872287642] |0.0       |0        |
|[1.5147691730861013,-1.5147691730861013]|[0.8197669228469342,0.1802330771530658]  |0.0       |0        |
|[-10.111472793343713,10.111472793343713]|[4.060930203363309E-5,0.9999593906979662]|1.0       |1        |
|[1.59997753980792,-1.59997753980792]    |[0.8

Note that some of the **rawPrediction** and **probability** values that were previously predicted as 0 are now predicted as 1

In [8]:
# Recalculate confusion matrix
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())
metrics2 = spark.createDataFrame([
 ("TP", tp2),
 ("FP", fp2),
 ("TN", tn2),
 ("FN", fn2),
 ("Precision", tp2 / (tp2 + fp2)),
 ("Recall", tp2 / (tp2 + fn2))],["metric", "value"])
metrics2.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           41751.0|
|       FP|             135.0|
|       TN|          649538.0|
|       FN|          119311.0|
|Precision|0.9967769660507091|
|   Recall|0.2592231563000584|
+---------+------------------+

Note that there are now more True Positives and less False Negatives, and Recall has improved. By changing the discrimination threshold, the model now gets more predictions correct - though it's worth noting that the number of False Positives has also increased.